In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [6]:
card_details={}
card_details['cat2']=[]
card_details['cat3']=[]
card_details['관광지명']=[]
card_details['지역명']=[]

raw_url_1='http://api.visitkorea.or.kr/openapi/service/rest/KorService/areaBasedList?ServiceKey=0Pi8vu%2F%2BRtATNDBh0aSxxzznYgmCjRysSSN0BDpRsCrf99%2Bbt1E551bGK5eLkmfGtE%2F1c0a80xT0tB83Ql%2FAWw%3D%3D&contentTypeId=&areaCode=&sigunguCode=&cat1=&cat2=&cat3=&listYN=Y&MobileOS=ETC&MobileApp=TourAPI3.0_Guide&arrange=A&numOfRows=12&pageNo={}'
raw_url_2='http://api.visitkorea.or.kr/openapi/service/rest/KorService/areaBasedList?ServiceKey=fbUkx4efUp1hBNUpOELVu4YkG99gnny98yOgf7pbx73Mr0ClesVNa57d%2B%2BbjUnySsp0BiGFCOcJ6EEiLLW4mbg%3D%3D&contentTypeId=&areaCode=&sigunguCode=&cat1=&cat2=&cat3=&listYN=Y&MobileOS=ETC&MobileApp=TourAPI3.0_Guide&arrange=A&numOfRows=12&pageNo={}'
raw_url_3='http://api.visitkorea.or.kr/openapi/service/rest/KorService/areaBasedList?ServiceKey=bOBLhS9Vu2WkmBHPaan7z2SPnJygZNdrqkL2YIAWOB4c8KFMWnIWN02ljXI9CNM%2FFfCOQWxpn7fV3Vm%2F%2FdPt%2BA%3D%3D&contentTypeId=&areaCode=&sigunguCode=&cat1=&cat2=&cat3=&listYN=Y&MobileOS=ETC&MobileApp=TourAPI3.0_Guide&arrange=A&numOfRows=12&pageNo={}'

for n in range(2807):
    if n <= 999:
        url=raw_url_1.format(n+1)
    elif n <= 1999:
        url=raw_url_2.format(n+1)
    else:
        url=raw_url_3.format(n+1)
        
    response=requests.request('GET',url)
    data=BeautifulSoup(response.text, 'lxml-xml')
    cards_data=data.find_all('item')
    for card in cards_data:
        try:
            card_details['cat2'].append(card.find('cat2').string)
        except:
            card_details['cat2'].append('None')
        try:
            card_details['cat3'].append(card.find('cat3').string)
        except:
            card_details['cat3'].append('None')
        try:
            card_details['관광지명'].append(card.find('title').string)
        except:
            card_details['관광지명'].append('None')
        try:
            pattern=r'\s'
            result=re.split(pattern, card.find('addr1').string)
            area=result[0]+' '+result[1]
            card_details['지역명'].append(area)
        except:
            card_details['지역명'].append('None')
raw=pd.DataFrame(card_details)
raw

,cat2,cat3,관광지명,지역명
0,C0114,C01140001,‘바다여행의 종합 세트’ 사천으로 가자,None
1,None,None,‘이열치열’ 하러 대구로 떠나자!,None
2,C0112,C01120001,‘직지’의 고장이자 쉼과 여유의 휴양지,None
3,C0115,C01150001,‘충혼’의 수양벚꽃 마중하는 호젓한 꽃길,None
4,C0114,C01140001,"“동굴아, 여름을 부탁해”...그늘 찾아 떠나는 삼척여행",None
...,...,...,...,...
33679,B0201,B02011100,people(후암별장3030),서울특별시 용산구
33680,B0201,B02011100,pine view,경기도 수원시
33681,A0502,A05020200,santa 아리아,경기도 안성시
33682,A0305,A03050100,scg스포츠아카데미,서울특별시 은평구


In [3]:
df=pd.read_csv('C:/Users/82102/Desktop/문화 관광 project/관광타입분류.csv')
mid=df[['중분류 (cat2)','중분류']]
mid.rename(columns={'중분류 (cat2)':'cat2'}, inplace=True)
mid.drop_duplicates(['cat2'],inplace=True)
mid.reset_index(inplace=True)
mid=mid[['cat2','중분류']]

ss=df[['소분류 (cat3)','소분류']]
ss.rename(columns={'소분류 (cat3)':'cat3'}, inplace=True)
ss.drop_duplicates(['cat3'],inplace=True)
ss.reset_index(inplace=True)
ss=ss[['cat3','소분류']]

C:\Users\82102\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\82102\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [7]:
raw2=pd.merge(raw,mid,how='left', on='cat2')
raw3=pd.merge(raw2,ss,how='left', on='cat3')

In [8]:
classif=raw3[['관광지명','지역명','중분류','소분류']]
classif

,관광지명,지역명,중분류,소분류
0,‘바다여행의 종합 세트’ 사천으로 가자,None,힐링코스,힐링코스
1,‘이열치열’ 하러 대구로 떠나자!,None,NaN,NaN
2,‘직지’의 고장이자 쉼과 여유의 휴양지,None,가족코스,가족코스
3,‘충혼’의 수양벚꽃 마중하는 호젓한 꽃길,None,도보코스,도보코스
4,"“동굴아, 여름을 부탁해”...그늘 찾아 떠나는 삼척여행",None,힐링코스,힐링코스
...,...,...,...,...
33679,people(후암별장3030),서울특별시 용산구,숙박시설,게스트하우스
33680,pine view,경기도 수원시,숙박시설,게스트하우스
33681,santa 아리아,경기도 안성시,음식점,서양식
33682,scg스포츠아카데미,서울특별시 은평구,복합 레포츠,복합 레포츠


In [9]:
classif.to_csv('관광정보분류.csv', index=False, encoding='UTF-8-sig')

In [3]:
classif=pd.read_csv('관광정보분류.csv')

In [10]:
classif['중분류'].value_counts()

음식점       8730
역사관광지     4051
육상 레포츠    3528
자연관광지     3323
숙박시설      3266
문화시설      2225
체험관광지     1768
휴양관광지     1664
쇼핑        1404
축제         827
수상 레포츠     529
건축/조형물     472
힐링코스       396
가족코스       331
산업관광지      299
공연/행사      205
관광자원       154
도보코스       126
나홀로코스       96
복합 레포츠      76
레포츠소개       72
캠핑코스        36
항공 레포츠      23
맛코스         19
Name: 중분류, dtype: int64

In [4]:
classif[classif['지역명']=='경주시 산내면']

,관광지명,지역명,중분류,소분류
7572,동창천 청룡폭포(청룡인공폭포),경주시 산내면,자연관광지,폭포


In [18]:
classif[classif['중분류']=='체험관광지']

,관광지명,지역명,중분류,소분류
21,가고파 꼬부랑길 벽화마을,경상남도 창원시,체험관광지,이색거리
45,가락공방,충청북도 청주시,체험관광지,이색체험
70,가루매마을,경기도 양평군,체험관광지,농.산.어촌 체험
108,가뫼골마을,경상남도 진주시,체험관광지,농.산.어촌 체험
131,가산산성마을,경상북도 칠곡군,체험관광지,농.산.어촌 체험
...,...,...,...,...
33100,GB포유공방,전라남도 영광군,체험관광지,이색체험
33158,K-26,경기도 가평군,체험관광지,이색체험
33174,KT&G 상상마당 부산,부산광역시 부산진구,체험관광지,이색체험
33270,VRZONE 용산 아이파크몰점,서울특별시 용산구,체험관광지,이색체험


In [27]:
classif['지역명'].unique()

array(['None', '서울특별시 종로구', '강원도 고성군', '경기도 양평군', '서울특별시 용산구', '충청남도 태안군',
       '경기도 이천시', '충청남도 공주시', '부산광역시 부산진구', '전라남도 신안군', '충청북도 청주시',
       '충청남도 부여군', '경상북도 안동시', '전라남도 진도군', '경상남도 창원시', '강원도 평창군',
       '경상남도 통영시', '경상남도 산청군', '대전광역시 유성구', '경기도 양주시', '경기도 하남시',
       '전라북도 남원시', '경기도 여주시', '부산광역시 강서구', '인천광역시 중구', '경기 가평군',
       '경기도 남양주시', '서울특별시 송파구', '경상남도 거제시', '전라북도 전주시', '충청북도 제천시',
       '경기도 광주시', '경상남도 합천군', '경기도 파주시', '제주특별자치도 서귀포시', '강원도 홍천군',
       '전라남도 여수시', '전라남도 곡성군', '경기도 포천시', '경상북도 영천시', '전라북도 부안군',
       '강원 홍천군', '충청남도 서산시', '충청남도 예산군', '강원도 인제군', '서울특별시 구로구',
       '인천광역시 남동구', '강원도 정선군', '서울특별시 강동구', '전라남도 담양군', '서울특별시 영등포구',
       '전라남도 영광군', '전라남도 해남군', '경기도 광명시', '인천광역시 미추홀구', '대구광역시 동구',
       '대전광역시 중구', '제주특별자치도 제주시', '경기도 의왕시', '광주광역시 서구', '경상남도 진주시',
       '서울특별시 강북구', '서울특별시 서대문구', '서울특별시 마포구', '경기도 구리시', '인천광역시 부평구',
       '경기도 용인시', '경기도 평택시', '경상남도 사천시', '전라남도 완도군', '경상북도 칠곡군',
       '충청북도 옥천군', '전라남도 장성군', '경상남도 양산시', '충청북도 영

In [5]:
classif['지역명'].value_counts().head(100)

None            1107
강원도 평창군          834
서울특별시 종로구        772
제주특별자치도 제주시      754
제주특별자치도 서귀포시     671
                ... 
경상남도 진주시         119
인천광역시 연수구        117
전라북도 정읍시         113
충청남도 서천군         111
경상북도 청도군         111
Name: 지역명, Length: 100, dtype: int64